<a href="https://colab.research.google.com/github/Hriman-Mahanta/Flood_Image_Segmentation/blob/main/IP_Project_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam
from keras import backend as K
import os
import cv2
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define the input shape of the images
input_shape = (256, 256, 3)

# Define the number of filters for each convolutional layer
n_filters = 64

# Define the input layer
inputs = Input(input_shape)

In [ ]:
# Load the dataset

path="/content/drive/MyDrive/Refined_Images/DATA_HAR"

data = {}  # Create an empty dictionary to store image-mask filename pairs

for filename in os.listdir(path):
    if filename.endswith('vhr.tif'):
        img_filename = filename
        mask_filename = filename.replace('vhr.tif', 'mask_flood.tif')
        
        if os.path.exists(os.path.join(path, mask_filename)):
            data[img_filename] = mask_filename  # Store the image-mask filename pair in the dictionary
            
X = []  # Create an empty list to store images
Y = []  # Create an empty list to store masks

for img_filename, mask_filename in data.items():
    img = cv2.imread(os.path.join(path, img_filename), cv2.IMREAD_COLOR)
    img = cv2.resize(img, (256, 256))
    print(img.shape)
    X.append(img)
    
    mask = cv2.imread(os.path.join(path, mask_filename), cv2.IMREAD_COLOR)
    mask = cv2.resize(mask, (256, 256))
    print(mask.shape)
    Y.append(mask)


# Convert the lists to NumPy arrays
X = np.array(X)
Y = np.array(Y)

# Split the dataset into training, validation, and test sets
n_samples = X.shape[0]
idx = np.arange(n_samples)
np.random.shuffle(idx)
X_train = X[idx[:int(0.7*n_samples)]]
Y_train = Y[idx[:int(0.7*n_samples)]]
X_val = X[idx[int(0.7*n_samples):int(0.85*n_samples)]]
Y_val = Y[idx[int(0.7*n_samples):int(0.85*n_samples)]]
X_test = X[idx[int(0.85*n_samples):]]
Y_test = Y[idx[int(0.85*n_samples):]]


In [ ]:
# Define the contracting path of the UNet model
conv1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(n_filters*2, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(n_filters*2, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(n_filters*4, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(n_filters*4, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(n_filters*8, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(n_filters*8, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(n_filters*16, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(n_filters*16, (3, 3), activation='relu', padding='same')(conv5)

# Define the expansive path of the UNet model
up6 = Conv2DTranspose(n_filters*8, (2, 2), strides=(2, 2), padding='same')(conv5)
concat6 = concatenate([up6, conv4])
conv6 = Conv2D(n_filters*8, (3, 3), activation='relu', padding='same')(concat6)
conv6 = Conv2D(n_filters*8, (3, 3), activation='relu', padding='same')(conv6)

up7 = Conv2DTranspose(n_filters*4, (2, 2), strides=(2, 2), padding='same')(conv6)
concat7 = concatenate([up7, conv3])
conv7 = Conv2D(n_filters*4, (3, 3), activation='relu', padding='same')(concat7)
conv7 = Conv2D(n_filters*4, (3, 3), activation='relu', padding='same')(conv7)

up8 = Conv2DTranspose(n_filters*2, (2, 2), strides=(2, 2), padding='same')(conv7)
concat8 = concatenate([up8, conv2])
conv8 = Conv2D(n_filters*2, (3, 3), activation='relu', padding='same')(concat8)
conv8 = Conv2D(n_filters*2, (3, 3), activation='relu', padding='same')(conv8)

up9 = Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv8)
concat9 = concatenate([up9, conv1])
conv9 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(concat9)
conv9 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv9)

In [ ]:
# Define the output layer with sigmoid activation
outputs = Conv2D(3, (1, 1), activation='sigmoid')(conv9)

In [ ]:
def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1, 2, 3])
    union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3]) - intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
    dice = K.mean((2. * intersection + smooth) / (K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3]) + smooth), axis=0)
    return dice

In [ ]:
# Define the model with input and output layers
model = Model(inputs=[inputs], outputs=[outputs])

In [ ]:
# Define the loss function as binary cross-entropy and optimizer as Adam
model.compile(optimizer=Adam(lr=2e-3), loss='binary_crossentropy', metrics=['accuracy', dice_coef, iou_coef])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_val, Y_val))

Epoch 1/10
42/42 [==============================] - 124s 2s/step - loss: 0.6541 - accuracy: 0.3028 - dice_coef: 0.0399 - iou_coef: 0.0212 - val_loss: 0.2356 - val_accuracy: 0.2740 - val_dice_coef: 0.0406 - val_iou_coef: 0.0213
Epoch 2/10
42/42 [==============================] - 59s 1s/step - loss: 0.2258 - accuracy: 0.3781 - dice_coef: 0.0402 - iou_coef: 0.0211 - val_loss: 0.2271 - val_accuracy: 0.1754 - val_dice_coef: 0.0412 - val_iou_coef: 0.0217
Epoch 3/10
42/42 [==============================] - 59s 1s/step - loss: 0.2216 - accuracy: 0.2876 - dice_coef: 0.0409 - iou_coef: 0.0214 - val_loss: 0.2322 - val_accuracy: 0.4180 - val_dice_coef: 0.0509 - val_iou_coef: 0.0271
Epoch 4/10
42/42 [==============================] - 59s 1s/step - loss: 0.2222 - accuracy: 0.2699 - dice_coef: 0.0429 - iou_coef: 0.0226 - val_loss: 0.2291 - val_accuracy: 0.2669 - val_dice_coef: 0.0499 - val_iou_coef: 0.0265
Epoch 5/10
42/42 [==============================] - 59s 1s/step - loss: 0.2191 - accuracy: 0.23